#**Import Modules**

Selenium, Openpyxl, Regex, Files, Time

In [29]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.managed_default_content_settings.images": 2,
         "download_restrictions": 3}
chrome_options.add_experimental_option("prefs", prefs)
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [824 kB]
Get:15 http:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [0]:
from selenium.common.exceptions import *
from selenium.webdriver.common.action_chains import ActionChains

In [0]:
import time

In [0]:
import openpyxl

In [0]:
import regex as re

In [0]:
from google.colab import files

#**Mount Google Drive**

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [8]:
cd drive/My Drive/Colab Notebooks/Paper/Trade

/content/drive/My Drive/Colab Notebooks/Paper/Trade


In [9]:
ls

Crawler/  Data/  Paper-Authors/  Paper-Keywords/


#**Fetch all Paper Links**

In [11]:
MAINURL = "https://www.kci.go.kr/kciportal/po/search/poArtiSearList.kci?sereId=000047"
articleUrls = []

# access main page
wd.get(MAINURL)
time.sleep(0.5)

# fetch last page number
try:
    LASTPAGENO = wd.find_element_by_xpath("//a[@href]/img[@alt='마지막']").find_element_by_xpath("..").get_attribute("href")
    LASTPAGENO = int(LASTPAGENO[LASTPAGENO.index("(") + 1:LASTPAGENO.index(")")])
except NoSuchElementException:
    print("NO LAST PAGE!")

# fetch article urls
currPage = 1

while currPage <= LASTPAGENO:
    print("Reading page : " + str(currPage))

    # fetch all articles
    articles = wd.find_elements_by_xpath("//table[@class='list_paper no_title']/tbody/tr/td/p/label/a[@href]")
    for article in articles:
        articleUrls.append(article.get_attribute("href"))

    # go to next page
    try:
        TARGET = "//div[@class='pagenate']/a[@href=\'javascript:goPage(" + str(currPage + 1) + ")\']"
        wd.find_element_by_xpath(TARGET).click()
        currPage += 1
        time.sleep(0.8)
    except NoSuchElementException:
        break

len(articleUrls)

Reading page : 1
Reading page : 2
Reading page : 3
Reading page : 4
Reading page : 5
Reading page : 6
Reading page : 7
Reading page : 8
Reading page : 9
Reading page : 10
Reading page : 11
Reading page : 12
Reading page : 13
Reading page : 14
Reading page : 15
Reading page : 16
Reading page : 17
Reading page : 18
Reading page : 19
Reading page : 20
Reading page : 21
Reading page : 22
Reading page : 23
Reading page : 24
Reading page : 25
Reading page : 26
Reading page : 27
Reading page : 28
Reading page : 29
Reading page : 30
Reading page : 31
Reading page : 32
Reading page : 33
Reading page : 34
Reading page : 35
Reading page : 36
Reading page : 37
Reading page : 38
Reading page : 39
Reading page : 40
Reading page : 41
Reading page : 42
Reading page : 43
Reading page : 44
Reading page : 45
Reading page : 46
Reading page : 47
Reading page : 48
Reading page : 49
Reading page : 50
Reading page : 51
Reading page : 52
Reading page : 53
Reading page : 54
Reading page : 55
Reading page : 56
R

1293

#**Fetch and Write Data from each Volumes**

In [0]:
wb = openpyxl.Workbook()
ws = wb.active

In [0]:
TARGETPATH = "./Data/raw/Korean/"

In [31]:
 r = 2

 # write header
 ws.cell(row = 1, column = 1).value = 'journal'
 ws.cell(row = 1, column = 2).value = 'year'
 ws.cell(row = 1, column = 3).value = 'title'
 ws.cell(row = 1, column = 4).value = 'keywords'
 ws.cell(row = 1, column = 5).value = 'authors'

 for articleurl in articleUrls:
    # access volume page
    wd.get(articleurl)
    time.sleep(1)

    # fetch title
    try:
        title = wd.find_element_by_xpath("//p[@class='title']/span").get_attribute("innerText")
    except NoSuchElementException:
        continue
    #print(title)

    # fetch year
    try:
        yr = wd.find_element_by_xpath("//div[@class='journalInfo']/p[@class='vol']/a").get_attribute("innerText").strip()
        yr = yr[:yr.index(",")]
    except ValueError:
        continue
    except NoSuchElementException:
        continue
    #print(yr)
        
    # fetch authors
    authors = []
    a_tags = wd.find_elements_by_xpath("//div[@class='author']/a")
    for a in a_tags:
        author = a.get_attribute("innerText").strip()
        author = author[:author.index(" ")]
        authors.append(author)
    while ' ' in authors:
        authors.remove(' ')
    while '' in authors:
        authors.remove('')
    if(len(authors) == 0):
        continue
    #print(authors)

    # fetch keywords
    keywords = []
    try:
        h2_tag = wd.find_element_by_xpath("//div[@class='articleBody']/div[@class='box']/h2[contains(text(), '키워드')]")
        try:
            p_tag = h2_tag.find_element_by_xpath("../div/p")
            try:
                a_tags = h2_tag.find_elements_by_xpath("../div/div/a")
                keywords_candid_1 = p_tag.get_attribute("innerText").strip().split(", ")
                keywords_candid_2 = []
                for a in a_tags:
                    keywords_candid_2.append(a.get_attribute("innerText"))
                candid_1 = 0
                candid_2 = 0
                for kwd in keywords_candid_1:
                    if len(re.compile("[가-힣]+").findall(kwd)) > 0:
                        candid_1 += 1
                for kwd in keywords_candid_2:
                    if len(re.compile("[가-힣]+").findall(kwd)) > 0:
                        candid_2 += 1
                if candid_1 <= candid_2:
                    keywords = keywords_candid_1
                else:
                    keywords = keywords_candid_2
            except NoSuchElementException:
                keywords = p_tag.get_attribute("innerText").strip().split(", ")
        except NoSuchElementException:
            a_tags = h2_tag.find_elements_by_xpath("../div/div/a")
            for a in a_tags:
                keywords.append(a.get_attribute("innerText"))
    except NoSuchElementException:
        continue
    while ' ' in keywords:
        keywords.remove(' ')
    while '' in keywords:
        keywords.remove('')
    '''
    hanguel_idx = []
    for i in range(len(keywords)):
        if len(re.compile("[가-힣]+").findall(keywords[i])) > 0:
            hanguel_idx.append(i)
    hanguel_idx.reverse()
    for i in hanguel_idx:
        del keywords[i]
    '''
    if(len(keywords) == 0):
        continue
    #print(keywords)

    # write data to excel file
    print("Writing, " + title)
    ws.cell(row = r, column = 1).value = "Korea Trade Review"
    ws.cell(row = r, column = 2).value = yr
    ws.cell(row = r, column = 3).value = title
    for j in range(len(keywords)):
        ws.cell(row = r + j, column = 4).value = keywords[j]
    for j in range(len(authors)):
        ws.cell(row = r + j, column = 5).value = authors[j]
    r += max([len(keywords), len(authors)])

    if articleUrls.index(articleurl) % 10 == 0:
        print(str(articleUrls.index(articleurl) + 1) + " out of " + str(len(articleUrls)))

Writing, The Differences in the Selection of Outward FDI Locations between State- and Privately Owned Enterprises of China: Focusing on the Effects of Host Country Factors
1 out of 1293
Writing, 강압적 경제･통상 조치에 대한 분석과 남북한 경제 협력에의 시사점
Writing, 한국-유럽 국내 자동차부품의 운송루트 선정과 다중경로 구축에 관한 비교연구
Writing, A Study on the Principle of Good Faith in Korea : Concept and Application
Writing, SNA를 활용한 정기선사의 전략적 제휴 재편에 따른 항만 네트워크 변화 분석
Writing, How does FDI promote Economic Growth: Evidence from Mekong River Countries?
Writing, 미국의 우회덤핑방지제도와 회피방지제도에 대한 우리나라의 대응방안
Writing, 베트남에서 무역과 FDI가 소득불평등에 미친 영향
Writing, 국제물품매매계약에 있어서 국제규범들의 통일화에 관한 연구
Writing, ODA, FDI 및 교육비 지출이 경제성장에 미치는 영향: Doi Moi 이후의 베트남
Writing, 해외직접구매 소비자의 브랜드이미지와 구매의도 간 지각된가치, 가격민감도, 만족도의 구조적 관계 연구
11 out of 1293
Writing, 4차 산업혁명 시대 중국 패스트푸드점의 고객만족, 충성도와 행복감의 결정요인; 모바일주문 서비스의 편리성이 새로운 결정요인인가?
Writing, 제조물책임법 입증책임에 관한 연구
Writing, 일본의 한국산 수산물 수입규제 가능성 검토 - 한･일 방사능 분쟁을 중심으로 -
Writing, 동북아 오일허브 정책의 성과와 전망에 대한 연구
Writing, 무역거래에서 송금(T/T)방식의 결제 위험성에 영향

#**Save File**

In [0]:
wb.save(filename = TARGETPATH + "korea_trade_review.xlsx")

#**Download FIle**

In [0]:
files.download(TARGETPATH + "korea_trade_review.xlsx")